In [3]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from datetime import datetime
import numpy as np

# Make team names uniform between sheets

In [3]:
names_file ='/Users/stefanfeiler/Desktop/SmartBetter/SmartBetter/data/names.csv'

In [4]:
df = pd.read_csv(names_file)

In [5]:
mapping_dict = {key: value for key, value in zip(df['abbrev'], df['full_name'])}

In [ ]:
extra_data_file = '/Users/stefanfeiler/Desktop/SmartBetter/SmartBetter/data/mlb_extra_stats.csv'

In [ ]:
extra_df = pd.read_csv(extra_data_file)

In [ ]:
extra_df = extra_df.replace(mapping_dict)

In [1]:
extra_data_file_new = '/Users/stefanfeiler/Desktop/SmartBetter/SmartBetter/data/mlb_extra_stats_uniform_names.csv'

In [ ]:
extra_df.to_csv(extra_data_file_new)

# Step 2: Convert all times to MST

In [4]:
raw = pd.read_csv('/Users/stefanfeiler/Desktop/SmartBetter/SmartBetter/data/mlb_raw_full.csv')

/Users/stefanfeiler/Library/Python/3.9/lib/python/site-packages/IPython/core/interactiveshell.py:3505: DtypeWarning: Columns (90,92,94,96,110,112,114,116,118,120,122,124,126,128,130,132,134,136,138,140,142,144,146,148,150,152,154,156,158,160,162,164,166,168) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
# # This takes 2.5 minutes
time_columns = [col for col in raw.columns if 'time' in col]
raw[time_columns] = raw[time_columns].apply(lambda x: pd.to_datetime(x, format="%m/%d/%y %H:%M"))

In [6]:
#raw['commence_time'] = raw['commence_time'] - pd.Timedelta(hours=7)
raw[time_columns] = raw[time_columns]- pd.Timedelta(hours=7)

In [7]:
raw.head()

,game_id,commence_time,time_pulled,home_team,away_team,barstool_1_odds,barstool_1_time,barstool_2_odds,barstool_2_time,betclic_1_odds,...,wynnbet_2_odds,wynnbet_2_time,Unnamed: 161,Unnamed: 162,Unnamed: 163,Unnamed: 164,Unnamed: 165,Unnamed: 166,Unnamed: 167,Unnamed: 168
0,86dfc7d1616e06cd86ded2603130b8fb,2022-11-05 17:04:00,2022-11-05 16:55:00,Houston Astros,Philadelphia Phillies,1.66,2022-11-05 16:49:00,2.3,2022-11-05 16:49:00,1.68,...,2.39,2022-11-05 16:49:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,86dfc7d1616e06cd86ded2603130b8fb,2022-11-05 17:04:00,2022-11-05 16:45:00,Houston Astros,Philadelphia Phillies,1.66,2022-11-05 16:44:00,2.3,2022-11-05 16:44:00,1.68,...,2.39,2022-11-05 16:44:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,86dfc7d1616e06cd86ded2603130b8fb,2022-11-05 17:04:00,2022-11-05 16:40:00,Houston Astros,Philadelphia Phillies,1.66,2022-11-05 16:40:00,2.3,2022-11-05 16:40:00,1.68,...,2.39,2022-11-05 16:40:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,86dfc7d1616e06cd86ded2603130b8fb,2022-11-05 17:04:00,2022-11-05 16:35:00,Houston Astros,Philadelphia Phillies,1.66,2022-11-05 16:35:00,2.3,2022-11-05 16:35:00,1.68,...,2.39,2022-11-05 16:35:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,86dfc7d1616e06cd86ded2603130b8fb,2022-11-05 17:04:00,2022-11-05 16:30:00,Houston Astros,Philadelphia Phillies,1.66,2022-11-05 16:29:00,2.3,2022-11-05 16:29:00,1.68,...,2.39,2022-11-05 16:29:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Step 3: Make the same my_game_id values in each sheet

In [8]:
extra = pd.read_csv(extra_data_file_new)

In [9]:
extra['home_team_final'] = extra[['home_team', 'away_team']].min(axis=1)
extra['away_team_final'] = extra[['home_team', 'away_team']].max(axis=1)

In [10]:
raw['date'] = raw['commence_time'].copy().dt.strftime('%Y%m%d')
extra['date']= extra['date'].astype(str)

In [11]:
raw['commence_time']

0        2022-11-05 17:04:00
1        2022-11-05 17:04:00
2        2022-11-05 17:04:00
3        2022-11-05 17:04:00
4        2022-11-05 17:04:00
                 ...        
921709   2020-07-22 17:10:00
921710   2020-07-22 18:40:00
921711   2020-07-22 16:05:00
921712   2020-07-22 16:30:00
921713   2020-07-22 17:05:00
Name: commence_time, Length: 921714, dtype: datetime64[ns]

In [12]:
raw['my_id'] = raw['home_team'] + raw['away_team'] + raw['date']
extra['my_id'] = extra['home_team_final'] + extra['away_team_final'] + extra['date']

# Step 4: Make a complicated dict

In [13]:
# Make a dict that maps raw my_id to raw game_id and includes duplicates
mapping_dict = {}
for key, value in zip(raw['my_id'], raw['game_id']):
    if key in mapping_dict:
        mapping_dict[key].append(value)  # Append value to existing key's list
        mapping_dict[key] = list(set(mapping_dict[key]))
    else:
        mapping_dict[key] = [value]  

In [14]:
count = 0
for key, values in mapping_dict.items():
    if len(values) == 2:
        count+=1
print(count)

194


In [15]:
print(len(mapping_dict))

5861


In [16]:
extra['number_of_game_today'] = extra['number_of_game_today'].astype(str)

In [17]:
# Make a dict that maps extra my_id to the extra game_value (double header or not)
extra_dict = {}
count = 0
for key, value in zip(extra['my_id'], extra['number_of_game_today']):
    if key in extra_dict:
        extra_dict[key].append(value)
        extra_dict[key] = list(extra_dict[key])
    else:
        extra_dict[key] = [value]
print(len(extra_dict))

5703


In [18]:
# Make a dict that maps game_id's to commence_times
id_commence_dict = {key: value for key, value in zip(raw['game_id'], raw['commence_time'])}

In [19]:
# Go through each mapping_dict item
# if the length of values is 1, put it in the new dict with the suffix '_0'
# if the length of values is 2, put it in the new dict with the proper suffix
new_dict = {}
for key, values in mapping_dict.items():
    if len(values) == 1:
        new_dict[str(str(key) + '_0')] = values[0]
    elif len(values) == 2:
        if id_commence_dict[values[0]] < id_commence_dict[values[1]]:
            new_dict[str(str(key) + '_1')] = values[0]
            new_dict[str(str(key) + '_2')] = values[1]
        elif id_commence_dict[values[0]] > id_commence_dict[values[1]]:
            new_dict[str(str(key) + '_2')] = values[0]
            new_dict[str(str(key) + '_1')] = values[1]

In [20]:
extra['my_id_game'] = extra['home_team_final'] + extra['away_team_final'] + extra['date'] + '_' + extra['number_of_game_today']

In [21]:
my_id_game = extra['my_id_game'].tolist()

In [22]:
count = 0
new_list = []
for key, value in new_dict.items():
    if key in my_id_game:
        new_list.append(key)


In [23]:
count = 0
for item in new_list:
    if item in my_id_game:
        count +=1
print(count)

5650


In [24]:
# Takes about 2 minutes
raw['my_game_id_final'] = raw['game_id'].apply(lambda x: next((k for k, v in new_dict.items() if v == x), None))
raw['my_game_id_final']

0         Houston AstrosPhiladelphia Phillies20221105_0
1         Houston AstrosPhiladelphia Phillies20221105_0
2         Houston AstrosPhiladelphia Phillies20221105_0
3         Houston AstrosPhiladelphia Phillies20221105_0
4         Houston AstrosPhiladelphia Phillies20221105_0
                              ...                      
921709     Chicago White SoxMilwaukee Brewers20200722_0
921710     Los Angeles AngelsSan Diego Padres20200722_0
921711            Chicago CubsMinnesota Twins20200722_0
921712        Boston Red SoxToronto Blue Jays20200722_0
921713          Colorado RockiesTexas Rangers20200722_0
Name: my_game_id_final, Length: 921714, dtype: object

In [25]:
raw['winning_team'] = raw['my_game_id_final'].map(extra.set_index('my_id_game')['winning_team'])

raw['number_of_game_today'] = raw['my_game_id_final'].map(extra.set_index('my_id_game')['number_of_game_today'])

raw['day_of_week'] = raw['my_game_id_final'].map(extra.set_index('my_id_game')['day_of_week'])

raw['away_team'] = raw['my_game_id_final'].map(extra.set_index('my_id_game')['away_team'])
raw['away_team_league'] = raw['my_game_id_final'].map(extra.set_index('my_id_game')['away_team_league'])
raw['away_team_game_number'] = raw['my_game_id_final'].map(extra.set_index('my_id_game')['away_team_game_number'])

raw['home_team'] = raw['my_game_id_final'].map(extra.set_index('my_id_game')['home_team'])
raw['home_team_league'] = raw['my_game_id_final'].map(extra.set_index('my_id_game')['home_team_league'])
raw['home_team_game_number'] = raw['my_game_id_final'].map(extra.set_index('my_id_game')['home_team_game_number'])

raw['day_night'] = raw['my_game_id_final'].map(extra.set_index('my_id_game')['day_night'])
raw['park_id'] = raw['my_game_id_final'].map(extra.set_index('my_id_game')['park_id'])

In [26]:
raw_last_1000 = raw.tail(1000)
raw_last_1000.to_csv('/Users/stefanfeiler/Desktop/SmartBetter/SmartBetter/data/mlb_raw_final_last_1000.csv',index=False)

In [27]:
raw_first_1000 = raw.head(1000)
raw_first_1000.to_csv('/Users/stefanfeiler/Desktop/SmartBetter/SmartBetter/data/mlb_raw_final_first_1000.csv',index=False)

In [28]:
raw.to_csv('/Users/stefanfeiler/Desktop/SmartBetter/SmartBetter/data/mlb_raw_final_categories.csv',index=False)

In [29]:
raw.to_parquet('/Users/stefanfeiler/Desktop/SmartBetter/SmartBetter/data/mlb_raw_final_categories.parquet', index=False)

In [30]:
raw = pd.read_parquet('/Users/stefanfeiler/Desktop/SmartBetter/SmartBetter/data/mlb_raw_final_categories.parquet')

# DONE!!!!

In [31]:
# Some extra cleaning. We want to drop all columns where we don't know 

# BEFORE WE DO ANYTHING, WE NEED TO READ IN THE FULL RAW CSV AND SAVE THE TEAM_1 AND TEAM_2 COLUMNS BACK INTO THE FINAL DF

raw = pd.read_parquet('/Users/stefanfeiler/Desktop/SmartBetter/SmartBetter/data/mlb_raw_final_categories.parquet')

In [32]:
raw_with_alpha_teams = pd.read_csv('/Users/stefanfeiler/Desktop/SmartBetter/SmartBetter/data/mlb_raw_full.csv')

/Users/stefanfeiler/Library/Python/3.9/lib/python/site-packages/IPython/core/interactiveshell.py:3505: DtypeWarning: Columns (90,92,94,96,110,112,114,116,118,120,122,124,126,128,130,132,134,136,138,140,142,144,146,148,150,152,154,156,158,160,162,164,166,168) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [33]:
raw_home_away = pd.read_parquet('/Users/stefanfeiler/Desktop/SmartBetter/SmartBetter/data/mlb_raw_final_categories.parquet')

In [34]:
print(len(raw_with_alpha_teams))

921714


In [35]:
print(len(raw_home_away))

921714


In [36]:
raw_home_away['team_1'] = raw_with_alpha_teams['home_team']
raw_home_away['team_2'] = raw_with_alpha_teams['away_team']

In [37]:
raw_home_away.to_parquet('/Users/stefanfeiler/Desktop/SmartBetter/SmartBetter/data/mlb_raw_final_categories.parquet', index=False)

# More recent data

In [1]:
# just found some more recent data. Have to process 2023 season up through the current date rn....

In [6]:
names_file ='/Users/stefanfeiler/Desktop/SmartBetter/SmartBetter/data/names.csv'

In [7]:
df = pd.read_csv(names_file)

In [8]:
mapping_dict = {key: value for key, value in zip(df['abbrev'], df['full_name'])}

In [9]:
extra_data_file = '/Users/stefanfeiler/Desktop/SmartBetter/SmartBetter/data/2023SKED.csv'

In [10]:
extra_df = pd.read_csv(extra_data_file)

In [11]:
extra_df = extra_df.replace(mapping_dict)

In [18]:
extra_df.to_csv('/Users/stefanfeiler/Desktop/SmartBetter/SmartBetter/data/2023SKED_uniform_names.csv')

In [19]:
# Nevermind! We don't have the scores.....